In [ ]:
using JSON

# --- helper functions ---
function get_energy(id)
    if id == "PW_0_0_0"
        return "cc_fsol"
    else
        return "cc_fsol_twasp_interpolated"
    end
end

function get_base_persistence_weights(id)
    simulation_setups = Dict(
        "PW_0_0_0" => [0.0, 0.0, 0.0],
        "PW_1_M04_M09" => [1.0, -0.4, -0.9],
        "PW_1_M07_M07" => [1.0, -0.7, -0.7],
        "PW_1_0_0" => [1.0, 0.0, 0.0],
        "PW_1_M1_1" => [1.0, -1.0, 1.0]
    )
    return simulation_setups[id]
end

function get_perturbation_and_initialization(energy, mol_type, n_mol)
    if mol_type == "hard_sphere"
        return "single_random_only_translation", "random_only_translation"
    else
        if occursin("cc", energy) && n_mol > 2
            return "single_random_get_index", "random"
        else
            return "single_random", "random"
        end
    end
end

# --- Simulation Parameters ---
mol_type = "6r7m"
n_mol = 2
simulation_time_minutes = 14 * 60.0

delaunay_eps = 100.0
overlap_jump = 0.0
overlap_slope = 1.1
rs = 1.4
η = 0.3665
exact_delaunay = false

T_search_runs = 0
T_search_time = 0.0

iterations = 1
algorithm = "rwm"
sa_level = [0.0] 

id = "PW_1_M07_M07"
energy = get_energy(id)

bounds = 60.0 + n_mol * 30.0 
perturbation, initialization = get_perturbation_and_initialization(energy, mol_type, n_mol)

σ_r = 0.2
σ_t = 1.25

persistence_weights = get_base_persistence_weights(id)
for temperature in [2.0, 3.0, 4.0, 5.0, 6.0]
    for scale in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        comment = "$(id)_MU_$(Int(round(scale * 10)))_T_$(Int(round(temperature * 10)))"
        comment = replace(comment, " " => "_")
        simulations_per_combination = sims_todo[(temperature, scale)] > 0 ? sims_todo[(temperature, scale)] : continue

        input_specifier = "$(n_mol)_$(mol_type)_$(algorithm)_$(comment)"
        output_directory = "../Simulations/unsorted_output/$(input_specifier)/"
        
        # --- Create a list of configuration dictionaries ---
        all_configs = []
        for i in 1:simulations_per_combination
            config = Dict(
                "name" => string(i),
                "x_init" => Float64[], # Use an empty array for JSON
                "temperature" => temperature,
                "algorithm" => algorithm,
                "sa_level" => sa_level,
                "T_search_runs" => T_search_runs,
                "T_search_time" => T_search_time,
                "rs" => rs,
                "η" => η,
                "mu" => scale,
                "σ_t" => σ_t,
                "σ_r" => σ_r,
                "overlap_jump" => overlap_jump,
                "overlap_slope" => overlap_slope,
                "bounds" => bounds,
                "persistence_weights" => persistence_weights,
                "n_mol" => n_mol,
                "mol_type" => mol_type,
                "energy" => energy,
                "perturbation" => perturbation,
                "initialization" => initialization,
                "output_directory" => output_directory,
                "delaunay_eps" => delaunay_eps,
                "exact_delaunay" => exact_delaunay,
                "comment" => comment,
                "mode" => "time",
                "simulation_time_minutes" => simulation_time_minutes,
                "iterations" => iterations
            )
            push!(all_configs, config)
        end

        # --- Write the list of configurations to a single JSON file ---
        config_path = "../configs/$(input_specifier)_config.json"
        open(config_path, "w") do f
            JSON.print(f, all_configs, 4) # Use an indent of 4 for readability
        end

        total_simulations = length(all_configs)
        total_time_needed = simulation_time_minutes + (T_search_runs * T_search_time)
        hours = Int(round(total_time_needed / 60.0))
        days = hours ÷ 24
        remaining_hours = hours % 24
        remaining_hours_string = remaining_hours < 10 ? "0$(remaining_hours)" : string(remaining_hours)
        buffer_time_string = total_time_needed < 5 ? "0$(Int(round(total_time_needed))+2)" : "30"

        mem_per_cpu = "1G" # Memory per CPU
        if n_mol >= 3
            mem_per_cpu = "1500M" # Memory per CPU for 3-molecule simulations
        end

        # --- Create the corresponding SLURM script ---
        job_script_path = "../$(input_specifier)_script.job"
        open(job_script_path, "w") do io
            println(io, "#!/bin/bash")
            println(io, "#SBATCH --job-name=SolvationSimulations")
            println(io, "#SBATCH --time=0$(days)-$(remaining_hours_string):$(buffer_time_string)")
            println(io, "#SBATCH --ntasks=1")
            println(io, "#SBATCH --cpus-per-task=1")
            println(io, "#SBATCH --mem-per-cpu=$(mem_per_cpu)")
            println(io, "#SBATCH --array=1-$(total_simulations)")
            println(io, "#SBATCH --chdir=/work/spirandelli/MorphoMolHPC/")
            println(io, "#SBATCH -o ./job_log/$(input_specifier)/%a.out # STDOUT")
            println(io, "")
            println(io, "export http_proxy=http://proxy2.uni-potsdam.de:3128 #Setting proxy, due to lack of Internet on compute nodes.")
            println(io, "export https_proxy=http://proxy2.uni-potsdam.de:3128")
            println(io, "")
            println(io, "module purge")
            println(io, "source ../oineus_venv/bin/activate")    
            println(io, "module load devel/CMake/3.27.6-GCCcore-13.2.0")
            println(io, "module load devel/Boost/1.83.0-GCC-13.2.0")
            println(io, "module load lang/Julia/1.7.3-linux-x86_64")    
            println(io, "")
            println(io, "# Define the path to the JSON config file")
            println(io, "CONFIG_JSON=\"hpc_scripts/configs/$(input_specifier)_config.json\"")
            println(io, "")
            println(io, "# Call the Julia script with the JSON path and the task ID")
            println(io, "julia julia_scripts/generic_call_from_json.jl \"\$CONFIG_JSON\" \"\$SLURM_ARRAY_TASK_ID\"")
        end
    end
end